In [12]:
import numpy as np
import pandas as pd

In [15]:
def create_mempool(txn_number, pre_df, post_df, t=0):
    mean, sd = 1, .1
    pre_gasprice = np.random.normal(mean, sd, txn_number)
    post_gasprice = np.random.normal(mean, sd, txn_number)
    gas_used = np.random.normal(3, sd, txn_number*2)
    
    pre_ethvalue = [gas_used[i]*pre_gasprice[i] for i in range(txn_number)]
    post_ethvalue = [gas_used[i+txn_number]*post_gasprice[i] for i in range(txn_number)]

#     pre_mempool = [(pre_gasprice[i], values[i]) for i in range(txn_number)]
#     post_mempool = [(post_gasprice[i], values[i+txn_number]) for i in range(txn_number)]

    pre_df = pre_df.append(pd.DataFrame({
        'gas price': pre_gasprice,
        'gas used': gas_used[:txn_number],
        'value in eth': pre_ethvalue,
      	'time' : t
        }), ignore_index=True)

    post_df = post_df.append(pd.DataFrame({
        'gas price': post_gasprice,
        'gas used': gas_used[txn_number:],
        'value in eth': post_ethvalue,
        'time': t
        }), ignore_index=True)
    
    pre_df = pre_df.sort_values(by=['gas price'], ascending=False).reset_index(drop=True)
    post_df = post_df.sort_values(by=['gas price'], ascending=False).reset_index(drop=True)
    

    
    return pre_df, post_df

def pre_fill_block(pre_df):
    block = []
    block_size = 0
    for i in range(len(pre_df)):
        txn = pre_df.iloc[i, :].to_list()
        if block_size + txn[1] > 12.5:
            break
        else:
            block += [txn]
            block_size += txn[1]
    
    pre_df = pre_df.iloc[i+1:,:]    
    
    return block, pre_df
        
def simulate(n, txn_number):
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    blocks = []
    block_data = pd.DataFrame()
    

    pre_data, post_data = create_mempool(txn_number, df1, df2)
    mempools = [pre_data]
    
    for i in range(n):
        filled_block = pre_fill_block(pre_data)
        blocks += [filled_block[0]]
        block_data = block_data.append(pd.DataFrame({
            'number of txns': [len(filled_block[0])],
            'total gas': [sum([filled_block[0][i][1] for i in range(len(filled_block[0]))])],
            'total revenue': [sum([filled_block[0][i][2] for i in range(len(filled_block[0]))])]
        }), ignore_index=True)
        pre_data = filled_block[1]
#         print(pre_data)
        mempools.append(filled_block[1])
        pre_data, post_data = create_mempool(txn_number, pre_data, post_data,i+1)
        
    return blocks, pre_data, block_data, mempools
    

In [16]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

s = simulate(10, 10)
for i in range(len(s[3])):
    print(s[3][i])

   gas price  gas used  value in eth  time
0   1.177658  2.873458      3.383951     0
1   1.138186  3.027030      3.445324     0
2   1.048299  2.892814      3.032536     0
3   1.047939  3.059256      3.205915     0
4   1.045785  3.101625      3.243632     0
5   1.025796  3.196645      3.279106     0
6   1.021152  3.023742      3.087700     0
7   1.011299  3.084542      3.119394     0
8   0.981297  2.973238      2.917629     0
9   0.863891  3.099864      2.677946     0
   gas price  gas used  value in eth  time
5   1.025796  3.196645      3.279106     0
6   1.021152  3.023742      3.087700     0
7   1.011299  3.084542      3.119394     0
8   0.981297  2.973238      2.917629     0
9   0.863891  3.099864      2.677946     0
    gas price  gas used  value in eth  time
5    1.011299  3.084542      3.119394     0
6    0.991531  3.214997      3.187768     1
7    0.981297  2.973238      2.917629     0
8    0.972953  3.048466      2.966015     1
9    0.929224  2.904893      2.699296     1
10   